In [1]:
import os
import yfinance as yf
from datetime import datetime
import pandas as pd
import pytz
import logging

import matplotlib.pyplot as plt

In [2]:
import numpy as np
import plotly.graph_objects as go
from PIL import Image

In [3]:
loaded = np.load('MSFT_test.npz')

In [4]:
dta_x = loaded['a']

In [5]:
dta_x.shape

(500, 700, 8568)

In [6]:
dta_y = loaded['b']
dta_y.shape

(8568,)

## Pytorch Machine Learning Shit

In [7]:
from ResNet_CNN import *

In [8]:
import math as m
import torch
from torch.nn import Linear, ReLU, Conv1d, Conv2d, Flatten, Sequential, CrossEntropyLoss, MSELoss, MaxPool1d, MaxPool2d, Dropout, BatchNorm1d, BatchNorm2d

from torch.optim import Adam
from torch import nn
import torchvision

In [9]:
from functools import partial
from collections import OrderedDict

In [10]:
import torchvision.models as models

In [11]:
from torchsummary import summary

In [12]:
model = res_conv1(1, 500)

In [13]:
print(model)

ResNet(
  (encoder): ResNetEncoder(
    (gate): Sequential(
      (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (blocks): ModuleList(
      (0): ResNetLayer(
        (blocks): Sequential(
          (0): ResNetBottleNeckBlock(
            (blocks): Sequential(
              (0): Sequential(
                (0): Conv2dAuto(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (1): ReLU(inplace=True)
              (2): Sequential(
                (0): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [14]:
summary(model.cuda(), (1,64,64))

------------------------------------------------------------------------------------------
Layer (type:depth-idx)                   Output Shape              Param #
├─ResNetEncoder: 1-1                     [-1, 2048, 2, 2]          --
|    └─Sequential: 2-1                   [-1, 64, 16, 16]          --
|    |    └─Conv2d: 3-1                  [-1, 64, 32, 32]          3,136
|    |    └─BatchNorm2d: 3-2             [-1, 64, 32, 32]          128
|    |    └─ReLU: 3-3                    [-1, 64, 32, 32]          --
|    |    └─MaxPool2d: 3-4               [-1, 64, 16, 16]          --
├─ResNetDecoder: 1-2                     [-1, 1]                   --
|    └─AdaptiveAvgPool2d: 2-2            [-1, 2048, 1, 1]          --
Total params: 9,086,815
Trainable params: 9,086,815
Non-trainable params: 0
------------------------------------------------------------------------------------------
Input size (MB): 0.02
Forward/backward pass size (MB): 1.00
Params size (MB): 34.66
Estimated Total Siz

------------------------------------------------------------------------------------------
Layer (type:depth-idx)                   Output Shape              Param #
├─ResNetEncoder: 1-1                     [-1, 2048, 2, 2]          --
|    └─Sequential: 2-1                   [-1, 64, 16, 16]          --
|    |    └─Conv2d: 3-1                  [-1, 64, 32, 32]          3,136
|    |    └─BatchNorm2d: 3-2             [-1, 64, 32, 32]          128
|    |    └─ReLU: 3-3                    [-1, 64, 32, 32]          --
|    |    └─MaxPool2d: 3-4               [-1, 64, 16, 16]          --
├─ResNetDecoder: 1-2                     [-1, 1]                   --
|    └─AdaptiveAvgPool2d: 2-2            [-1, 2048, 1, 1]          --
Total params: 9,086,815
Trainable params: 9,086,815
Non-trainable params: 0
------------------------------------------------------------------------------------------
Input size (MB): 0.02
Forward/backward pass size (MB): 1.00
Params size (MB): 34.66
Estimated Total Siz

## Putting everything together

In [15]:
X = dta_x[:,:,568:768]

In [16]:
Y = dta_y[568:768]

In [58]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [17]:
import torchvision.transforms as transforms

In [18]:
trans = transforms.Compose([transforms.ToTensor(),
                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [20]:
from torch.autograd import Variable
from torch.optim import Adam, SGD

In [32]:
X = X.reshape((-1, 500, 700)).astype(np.float32)
Y = Y.reshape((-1, 1)).astype(np.float32)

In [33]:
X.shape

(200, 500, 700)

In [34]:
Y.shape

(200, 1)

In [35]:
train_X, val_X, train_Y, val_Y = train_test_split(X, Y, test_size=0.1, random_state=42)

In [36]:
(train_X.shape, train_Y.shape), (val_X.shape, val_Y.shape)

(((180, 500, 700), (180, 1)), ((20, 500, 700), (20, 1)))

In [37]:
train_X = train_X.reshape(-1, 1, 500, 700)
train_X = torch.from_numpy(train_X)

train_Y = train_Y.astype(int)
train_Y = torch.from_numpy(train_Y)

In [38]:
val_X =  val_X.reshape(-1, 1, 500, 700)
val_X = torch.from_numpy(val_X)

val_Y = val_Y.astype(int)
val_Y = torch.from_numpy(val_Y)

In [39]:
model = res_conv1(1, 500)

In [40]:
lr = 0.01
optimizer = Adam(model.parameters(), lr=lr)

criterion = CrossEntropyLoss()

Don't run because CUDA is out of memory

In [72]:
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

ResNet(
  (encoder): ResNetEncoder(
    (gate): Sequential(
      (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (blocks): ModuleList(
      (0): ResNetLayer(
        (blocks): Sequential(
          (0): ResNetBottleNeckBlock(
            (blocks): Sequential(
              (0): Sequential(
                (0): Conv2dAuto(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (1): ReLU(inplace=True)
              (2): Sequential(
                (0): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [41]:
def train(epochs):
    model.train()
    tr_loss = 0
    # dataset
    x_train, y_train = Variable(train_X), Variable(train_Y)
    x_val, y_val = Variable(val_X), Variable(val_Y)
    '''
    if torch.cuda.is_available():
        x_train = x_train.to(device, dtype=torch.float32)
        y_train = y_train.to(device, dtype=torch.float32)
        x_val = x_val.to(device, dtype=torch.float32)
        y_val = y_val.to(device, dtype=torch.float32)
      '''
    optimizer.zero_grad()
    
    output_train = model(x_train)
    output_val = model(x_val)
    
    loss_train = criterion(output_train, y_train)
    loss_val = criterion(output_val, y_val)
    
    train_losses.append(loss_train)
    val_losses.append(loss_val)
    
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    
    if epochs % 2 == 0:
        print('Epoch: ', epochs+1, '\t', 'train loss: ', loss_train, '\t', 'val loss: ', loss_val)

In [42]:
n_epochs = 10
train_losses = []
val_losses = []

for epochs in range(n_epochs):
    train(epochs)

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 4032000000 bytes. Buy new RAM!
